In [6]:
import os
import librosa
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define the path to your dataset folder
dataset_path = "D:\Dil\S.P.I.T Semester 6\Mini project\dataset"

# Define the categories and the corresponding number of audio files
categories = ["belly_pain", "burping", "discomfort", "hungry", "tired"]
num_files = [16, 8, 27, 382, 24]

# Define the parameters for MFCC extraction
n_mfcc = 13
pre_emphasis = 0.97

# Define the classifiers to evaluate
classifiers = [GaussianNB(), RandomForestClassifier(n_estimators=100), KNeighborsClassifier(n_neighbors=3)]

# Define the lists to store the mean MFCC features and labels for each category
mfcc_features = []
labels = []

# Loop through each category folder and extract the mean MFCC features for each audio file
for i, category in enumerate(categories):
    for j in range(num_files[i]):
        # Load audio file
        audio_file = os.path.join(dataset_path, category, f"{category} ({j+1}).wav")
        signal, sr = librosa.load(audio_file, sr=16000)

        # Remove leading and trailing silence
        signal, _ = librosa.effects.trim(signal)

        # Normalize the signal
        signal = librosa.util.normalize(signal)

        # Apply pre-emphasis
        emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

        # Apply Hamming window
        hamming_window = np.hamming(len(emphasized_signal))
        emphasized_signal = emphasized_signal * hamming_window

        # Compute short-time zero crossing rate and short-time energy
        frame_length = 0.025 # seconds
        frame_stride = 0.01 # seconds
        hop_length = int(frame_stride * sr)
        win_length = int(frame_length * sr)
        zero_crossings = librosa.feature.zero_crossing_rate(emphasized_signal, pad=False, frame_length=win_length, hop_length=hop_length).sum()
        energy = np.sum(np.power(emphasized_signal, 2))

        # Compute MFCC features
        mfcc = librosa.feature.mfcc(y=emphasized_signal, sr=sr, n_mfcc=n_mfcc)

        # Compute the mean of the MFCC features, zero crossing rate and energy
        mfcc_mean = np.mean(mfcc, axis=1)
        features = np.concatenate([mfcc_mean, [zero_crossings, energy]])

        # Append the features and the label to the corresponding lists
        mfcc_features.append(features)
        labels.append(category)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(mfcc_features, labels, test_size=0.2, random_state=42)

# Train and evaluate each classifier
best_acc = 0
best_classifier = None
for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {type(clf).__name__}: {acc}")
    if acc > best_acc:
        best_acc = acc
        best_classifier = clf


# Use the best classifier to predict the category of a new audio file
new_audio_file = "D:\Dil\S.P.I.T Semester 6\Mini project\dataset\hungry\hungry (12).wav"
signal, sr = librosa.load(new_audio_file, sr=16000)

# Remove leading and trailing silence
signal, _ = librosa.effects.trim(signal)

# Normalize the signal
signal = librosa.util.normalize(signal)

# Apply pre-emphasis
emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

# Apply Hamming window
hamming_window = np.hamming(len(emphasized_signal))
emphasized_signal = emphasized_signal * hamming_window

# Compute short-time zero crossing rate and short-time energy
frame_length = 0.025 # seconds
frame_stride = 0.01 # seconds
hop_length = int(frame_stride * sr)
win_length = int(frame_length * sr)
zero_crossings = librosa.feature.zero_crossing_rate(emphasized_signal, pad=False, frame_length=win_length, hop_length=hop_length).sum()
energy = np.sum(np.power(emphasized_signal, 2))

# Compute MFCC features
mfcc = librosa.feature.mfcc(y=emphasized_signal, sr=sr, n_mfcc=n_mfcc)

# Compute the mean of the MFCC features, zero crossing rate and energy
mfcc_mean = np.mean(mfcc, axis=1)
features = np.concatenate([mfcc_mean, [zero_crossings, energy]])

# Use the best classifier to predict the category of the new audio file
predicted_category = best_classifier.predict([features])[0]
print(f"Predicted category: {predicted_category}")

Accuracy for GaussianNB: 0.7391304347826086
Accuracy for RandomForestClassifier: 0.782608695652174
Accuracy for KNeighborsClassifier: 0.7391304347826086
Predicted category: hungry
